In [1]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from isodate import parse_duration
import pyspark.sql.types as t
from typing import Tuple, List
import os

In [2]:
@f.udf(returnType=t.FloatType())
def converter_minutos(tempo: str) -> float:
    """Converte para o total de seguntos

    Args:
        tempo (str): tempo

    Returns:
        int: total segundos
    """
    duracao = parse_duration(tempo)
    total_segundos = duracao.total_seconds() 
    total_minutos = total_segundos / 60
   
    return round(total_minutos, 2)


In [3]:
spark = SparkSession.builder.appName('etl_trends_ouro').getOrCreate()
spark

24/02/24 21:42:57 WARN Utils: Your hostname, rodrigo-nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface wlp8s0)
24/02/24 21:42:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/24 21:42:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def criar_particao_ouro(dataframe: DataFrame, nome_arquivo: str ):
    caminho_datalake_ouro = '../dados/ouro'
    dataframe.write. \
                parquet(os.path.join(caminho_datalake_ouro, nome_arquivo),  )

In [5]:
def transformacao_dataframe_trends(dataframe_transform: DataFrame):
    dataframe_transform = dataframe_transform.select(
        'data_extracao',
        'DATA_PUBLICACAO',
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'ID_VIDEO',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     )
    dataframe_transform  = dataframe_transform.na.drop('all')
    dataframe_transform = dataframe_transform.withColumn("data_hora_extracao", f.col('data_extracao'))
    dataframe_transform = dataframe_transform.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd" ))
    dataframe_transform = dataframe_transform.withColumn('DURACAO_VIDEO_MINUTOS', converter_minutos(f.col('DURACAO_VIDEOS')))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_LIKES', f.col('TOTAL_LIKES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_FAVORITOS', f.col('TOTAL_FAVORITOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_COMENTARIOS', f.col('TOTAL_COMENTARIOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_CARACTERE_VIDEO', f.length('TITULO_VIDEO'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_TAGS', f.when(f.size('TAGS') > 0, f.size('TAGS')).otherwise(0))
    dataframe_transform = dataframe_transform.withColumn('HORA_EXTRACAO', f.hour("data_hora_extracao"))
    dataframe_transform = dataframe_transform.withColumn(
        'TURNO_EXTRACAO',
        (
            f.when(
                (dataframe_transform.HORA_EXTRACAO >=6) & (dataframe_transform.HORA_EXTRACAO < 12), 'Manhã')
            .when((dataframe_transform.HORA_EXTRACAO >= 12) & (dataframe_transform.HORA_EXTRACAO < 18), 'Tarde')
            .otherwise('Noite')
        )
    )
    dataframe_transform = dataframe_transform.withColumn(
        'INDICE_TURNO_EXTRACAO',
        (
            f.when(
                (dataframe_transform.TURNO_EXTRACAO == 'Manhã'), 1)
            .when((dataframe_transform.TURNO_EXTRACAO == 'Tarde'), 2)
            .otherwise(3)
        )
    )
    return dataframe_transform
  

In [6]:
def transformacao_dataframe(dataframe_transform: DataFrame):
    dataframe_transform = dataframe_transform.select(
        'data_extracao',
        'DATA_PUBLICACAO',
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'ID_VIDEO',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     )
    dataframe_transform  = dataframe_transform.na.drop('all')
    dataframe_transform = dataframe_transform.withColumn("data_hora_extracao", f.col('data_extracao'))
    dataframe_transform = dataframe_transform.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd" ))
    dataframe_transform = dataframe_transform.withColumn('DURACAO_VIDEO_MINUTOS', converter_minutos(f.col('DURACAO_VIDEOS')))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_LIKES', f.col('TOTAL_LIKES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_FAVORITOS', f.col('TOTAL_FAVORITOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_COMENTARIOS', f.col('TOTAL_COMENTARIOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_CARACTERE_VIDEO', f.length('TITULO_VIDEO'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_TAGS', f.when(f.size('TAGS') > 0, f.size('TAGS')).otherwise(0))
    dataframe_transform = dataframe_transform.withColumn('HORA_EXTRACAO', f.hour("data_hora_extracao"))
    dataframe_transform = dataframe_transform.withColumn(
        'TURNO_EXTRACAO',
        (
            f.when(
                (dataframe_transform.HORA_EXTRACAO >=6) & (dataframe_transform.HORA_EXTRACAO < 12), 'Manhã')
            .when((dataframe_transform.HORA_EXTRACAO >= 12) & (dataframe_transform.HORA_EXTRACAO < 18), 'Tarde')
            .otherwise('Noite')
        )
    )
    dataframe_transform = dataframe_transform.withColumn(
        'INDICE_TURNO_EXTRACAO',
        (
            f.when(
                (dataframe_transform.TURNO_EXTRACAO == 'Manhã'), 1)
            .when((dataframe_transform.TURNO_EXTRACAO == 'Tarde'), 2)
            .otherwise(3)
        )
    )
    return dataframe_transform
  

In [7]:
@f.udf(returnType=t.StringType())
def traduzir_dia_da_semana(dia):
    traducao = {
        'Sunday': 'Domingo',
        'Monday': 'Segunda-feira',
        'Tuesday': 'Terça-feira',
        'Wednesday': 'Quarta-feira',
        'Thursday': 'Quinta-feira',
        'Friday': 'Sexta-feira',
        'Saturday': 'Sábado'
    }
    return traducao.get(dia, dia)

In [8]:
@f.udf(returnType=t.IntegerType())
def indice_semana(dia):
    traducao = {
        'Domingo': 1,
        'Segunda-feira': 2,
        'Terça-feira': 3,
        'Quarta-feira': 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado': 7
    }
    return traducao.get(dia, dia)

In [9]:
df_estatisticas_gerais = spark.read.parquet('../dados/prata/arquivos_assuntos_trends.parquet')
df_estatisticas_gerais.columns

['data_extracao',
 'ID_VIDEO',
 'DATA_PUBLICACAO',
 'ID_CANAL',
 'NM_CANAL',
 'ID_CATEGORIA',
 'TITULO_VIDEO',
 'DESCRICAO',
 'TAGS',
 'DURACAO_VIDEOS',
 'TOTAL_VISUALIZACOES',
 'TOTAL_LIKES',
 'TOTAL_FAVORITOS',
 'TOTAL_COMENTARIOS']

In [10]:
df_estatisticas_gerais = transformacao_dataframe(dataframe_transform=df_estatisticas_gerais)
df_estatisticas_gerais.show()

+-------------+--------------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+---------------------+---------------------+----------+-------------+--------------+---------------------+
|data_extracao|     DATA_PUBLICACAO|            ID_CANAL|          NM_CANAL|ID_CATEGORIA|   ID_VIDEO|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS| data_hora_extracao|DURACAO_VIDEO_MINUTOS|TOTAL_CARACTERE_VIDEO|TOTAL_TAGS|HORA_EXTRACAO|TURNO_EXTRACAO|INDICE_TURNO_EXTRACAO|
+-------------+--------------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-----------

In [11]:
def construir_depara_canal(dataframe: DataFrame):
    dataframe.select(
        dataframe.ID_CANAL,
        dataframe.NM_CANAL,
        dataframe.ID_VIDEO,
        dataframe.TITULO_VIDEO
    ).distinct().sort('ID_CANAL').write.\
        parquet(f'../dados/ouro/depara_canal.parquet')


In [12]:
def obter_total_publicacao_video(dataframe : DataFrame) -> DataFrame:
   dataframe =  dataframe.select(
    'DATA_PUBLICACAO',
    'ID_CANAL',
    'NM_CANAL'
    ).withColumn(
       'DATA_PUBLICACAO', f.date_format('DATA_PUBLICACAO', 'yyyy-MM-dd')
    ) \
        .sort('ID_CANAL') \
        .withColumn(
            'SEMANA_TRADUZIDA', 
            traduzir_dia_da_semana(
                f.date_format('DATA_PUBLICACAO', 'EEEE')
            )
            ).distinct() \
                .groupBy('DATA_PUBLICACAO','SEMANA_TRADUZIDA', 'ID_CANAL', 'NM_CANAL'). \
                    count(). \
                        alias('TOTAL_VIDEOS'). \
                            withColumn('INDICE_SEMANA', indice_semana('SEMANA_TRADUZIDA')) \
                    .sort('INDICE_SEMANA')\
                        .select('DATA_PUBLICACAO','SEMANA_TRADUZIDA' ,'ID_CANAL', 'NM_CANAL', f.col('TOTAL_VIDEOS.COUNT').alias('TOTAL_VIDEOS'))       
   return dataframe

In [35]:
df_teste = df_estatisticas_gerais.select(
    'data_extracao',
    'ID_CANAL',
    'NM_CANAL',
    'ID_VIDEO',
    'TITULO_VIDEO',
    'TOTAL_VISUALIZACOES'
).orderBy([ 'data_extracao', 'ID_VIDEO'])

In [36]:
df_teste.show(truncate=False)

+-------------+------------------------+------------------+-----------+--------------------------------------------------------------------------------------------------+-------------------+
|data_extracao|ID_CANAL                |NM_CANAL          |ID_VIDEO   |TITULO_VIDEO                                                                                      |TOTAL_VISUALIZACOES|
+-------------+------------------------+------------------+-----------+--------------------------------------------------------------------------------------------------+-------------------+
|2024-01-17   |UC-OAb1sTjfO6cJYVdn_1GgA|RECORD EUROPA     |-7a68_XfpG4|Exclusivo: preso e nas vésperas de julgamento em Espanha, Daniel Alves tenta recuperar património|678169             |
|2024-01-17   |UC-OAb1sTjfO6cJYVdn_1GgA|RECORD EUROPA     |-7a68_XfpG4|Exclusivo: preso e nas vésperas de julgamento em Espanha, Daniel Alves tenta recuperar património|678155             |
|2024-01-17   |UC-OAb1sTjfO6cJYVdn_1GgA|RECOR

In [16]:
df_estatisticas_gerais.select(
    'TOTAL_LIKES', 'TOTAL_FAVORITOS', 'TOTAL_COMENTARIOS', 'TOTAL_VISUALIZACOES').show()

+-----------+---------------+-----------------+-------------------+
|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|TOTAL_VISUALIZACOES|
+-----------+---------------+-----------------+-------------------+
|       7555|              0|              440|             227954|
|      34934|              0|              624|            2278449|
|      13706|              0|              955|             227752|
|     140834|              0|             5644|            2277398|
|       4719|              0|             4192|              22747|
|       6964|              0|              296|             227454|
|      24725|              0|              370|             227411|
|       5703|              0|               84|             227012|
|      30277|              0|              937|             226943|
|       8383|              0|              347|             226875|
|      10301|              0|              317|             226514|
|     115207|              0|             6793| 

In [17]:
df_estatisticas_gerais.columns

['data_extracao',
 'DATA_PUBLICACAO',
 'ID_CANAL',
 'NM_CANAL',
 'ID_CATEGORIA',
 'ID_VIDEO',
 'TITULO_VIDEO',
 'DESCRICAO',
 'TAGS',
 'DURACAO_VIDEOS',
 'TOTAL_VISUALIZACOES',
 'TOTAL_LIKES',
 'TOTAL_FAVORITOS',
 'TOTAL_COMENTARIOS',
 'data_hora_extracao',
 'DURACAO_VIDEO_MINUTOS',
 'TOTAL_CARACTERE_VIDEO',
 'TOTAL_TAGS',
 'HORA_EXTRACAO',
 'TURNO_EXTRACAO',
 'INDICE_TURNO_EXTRACAO']

In [18]:
df_estatisticas_gerais_pandas = df_estatisticas_gerais.toPandas()
df_estatisticas_gerais_pandas

,data_extracao,DATA_PUBLICACAO,ID_CANAL,NM_CANAL,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,DESCRICAO,TAGS,DURACAO_VIDEOS,...,TOTAL_LIKES,TOTAL_FAVORITOS,TOTAL_COMENTARIOS,data_hora_extracao,DURACAO_VIDEO_MINUTOS,TOTAL_CARACTERE_VIDEO,TOTAL_TAGS,HORA_EXTRACAO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO
0,2024-01-20,2024-01-19T15:00:00Z,UCaEDMy0ItPSfPEvJf6wYIxw,Canal Ana Hickmann,22,-AHsFXxkxWQ,RECEITA DE PUDIM DE PÃO AMANHECIDO | COZINHAND...,"Oi, pessoal! Tudo bem? No vídeo de hoje, vou c...","[ana hickmann, canal ana hickmann, hoje em dia...",PT10M5S,...,7555.0,0,440.0,2024-01-20 11:01:47,10.08,58,19,11,Manhã,1
1,2024-01-22,2024-01-17T21:28:00Z,UCjaxmdb_gTWbz9XF5rzvgGg,Natan por Aí,24,pjTrKdCzihw,PARQUE AQUÁTICO de R$1 x R$ 1.000.000!,Qual parque aquático será que é o mais legal?\...,"[natan, natan por aí]",PT15M43S,...,34934.0,0,624.0,2024-01-22 11:05:09,15.72,38,2,11,Manhã,1
2,2024-01-19,2024-01-19T12:30:33Z,UCJIl6Ahgkfwg1DRYcg-kY3g,Mendrake,20,hQLdpblz3xQ,PROPOLIS vs MENDRAKE,mendrakecomercial@gmail.com\n\n🌐 Redes Sociais...,None,PT18M4S,...,13706.0,0,955.0,2024-01-19 21:57:25,18.07,20,0,21,Noite,3
3,2024-01-23,2024-01-17T18:17:36Z,UCWsDFcIhY2DBi3GB5uykGXA,IShowSpeed,20,A0u8rgz4qlA,Life In Brazil🇧🇷,MERCH HOODIE - https://www.ishowspeedstore.com...,"[ishowspeed, speed, speed brazil, ishowspeed b...",PT15M42S,...,140834.0,0,5644.0,2024-01-23 20:01:20,15.70,16,4,20,Noite,3
4,2024-01-19,2024-01-19T15:00:08Z,UC0hJfOjCQ15KQCWxgmbO0vA,Mc Caverinha,10,4DJ6aEwQm4U,"MC Caverinha, Wall Hein, Yokame - My Boo (Clip...","Esse é o videoclipe oficial da faixa ""My Boo"" ...","[MC Caverinha, Caverinha, Trap, Trap BR, Caver...",PT3M17S,...,4719.0,0,4192.0,2024-01-19 21:57:25,3.28,56,26,21,Noite,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,2024-01-21,2024-01-11T21:00:09Z,UCK3NkZPs0BurxabhtnSDa3g,Souzones,20,jn6kT6YBdOY,GOOSE GOOSE DUCK: MINHA REDENÇÃO APÓS A PIOR P...,👕 CAMISETAS do SOUZONES: https://www.youtube.c...,"[games, melhores momentos, momentos engraçados...",PT15M3S,...,30397.0,0,937.0,2024-01-21 10:59:44,15.05,99,5,10,Manhã,1
4396,2024-01-19,2024-01-12T15:00:07Z,UC0VOyT2OCBKdQhF3BAbZ-1g,ArianaGrandeVevo,10,eB6txyhHFG4,"Ariana Grande - yes, and? (official music video)","The official music video for ""yes, and?"" by Ar...","[Yes and music video, music video yes and, yes...",PT4M35S,...,1322088.0,0,79948.0,2024-01-19 21:57:26,4.58,48,14,21,Noite,3
4397,2024-01-20,2024-01-11T14:45:00Z,UCLjzb0-xUi_gfnZGd9ZgRwQ,Richard GameVlogs,20,2wpznW4qAu0,SOBREVIVI COMO SERPENTE GODZILLA DINOSSAURO 10...,,"[dinossauro, dinossauros, 100 dias, 100 dias a...",PT16M59S,...,5737.0,0,85.0,2024-01-20 21:52:28,16.98,98,31,21,Noite,3
4398,2024-01-18,2024-01-12T23:35:09Z,UCHNc0vOmeDCRLSzw5VgvNrQ,É o Vini,24,mnC4Y6w6ZYs,MARCANDO UM GOLAÇO DE CADA LINHA DO CAMPO!!!,Jesus te ama e que Deus abençoe você! \nEspero...,"[vini, é o vini, e o vini, futebol, desafios d...",PT11M14S,...,6259.0,0,106.0,2024-01-18 17:01:58,11.23,44,10,17,Tarde,2


In [21]:
df_estatisticas_gerais_pandas[['TOTAL_COMENTARIOS', 'TOTAL_VISUALIZACOES', 'TOTAL_LIKES']]

,TOTAL_COMENTARIOS,TOTAL_VISUALIZACOES,TOTAL_LIKES
0,440.0,227954,7555.0
1,624.0,2278449,34934.0
2,955.0,227752,13706.0
3,5644.0,2277398,140834.0
4,4192.0,22747,4719.0
...,...,...,...
4395,937.0,228647,30397.0
4396,79948.0,22835002,1322088.0
4397,85.0,228218,5737.0
4398,106.0,228163,6259.0


In [22]:
for chave, coluna in enumerate(df_estatisticas_gerais_pandas.columns):
    try:
        print(chave , coluna)
        print(df_estatisticas_gerais_pandas[coluna].unique())
        print()
    except:
        print('erro na coluna' ,coluna)

0 data_extracao
['2024-01-20' '2024-01-22' '2024-01-19' '2024-01-23' '2024-01-21'
 '2024-01-18' '2024-01-17']

1 DATA_PUBLICACAO
['2024-01-19T15:00:00Z' '2024-01-17T21:28:00Z' '2024-01-19T12:30:33Z'
 '2024-01-17T18:17:36Z' '2024-01-19T15:00:08Z' '2024-01-17T15:00:16Z'
 '2024-01-17T15:10:02Z' '2024-01-11T14:45:00Z' '2024-01-11T21:00:09Z'
 '2024-01-18T19:04:50Z' '2024-01-16T22:01:10Z' '2024-01-18T23:27:25Z'
 '2024-01-17T02:40:28Z' '2024-01-15T00:00:04Z' '2024-01-14T00:03:43Z'
 '2024-01-13T14:30:00Z' '2024-01-12T00:04:22Z' '2024-01-16T21:00:22Z'
 '2024-01-19T14:59:55Z' '2024-01-12T15:00:07Z' '2024-01-17T14:00:10Z'
 '2024-01-23T13:00:54Z' '2024-01-08T15:00:01Z' '2024-01-16T15:01:01Z'
 '2024-01-16T21:02:29Z' '2024-01-12T21:00:09Z' '2024-01-18T23:19:59Z'
 '2024-01-19T14:00:09Z' '2024-01-22T16:00:06Z' '2024-01-16T21:30:00Z'
 '2024-01-19T00:00:11Z' '2024-01-12T23:35:09Z' '2024-01-21T17:42:57Z'
 '2024-01-08T14:00:18Z' '2024-01-14T21:28:00Z' '2024-01-17T18:00:15Z'
 '2024-01-21T21:48:42Z' '2024-0

In [23]:
df_estatisticas_gerais_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_extracao          4400 non-null   object 
 1   DATA_PUBLICACAO        4400 non-null   object 
 2   ID_CANAL               4400 non-null   object 
 3   NM_CANAL               4400 non-null   object 
 4   ID_CATEGORIA           4400 non-null   object 
 5   ID_VIDEO               4400 non-null   object 
 6   TITULO_VIDEO           4400 non-null   object 
 7   DESCRICAO              4400 non-null   object 
 8   TAGS                   3661 non-null   object 
 9   DURACAO_VIDEOS         4400 non-null   object 
 10  TOTAL_VISUALIZACOES    4400 non-null   int32  
 11  TOTAL_LIKES            4288 non-null   float64
 12  TOTAL_FAVORITOS        4400 non-null   int32  
 13  TOTAL_COMENTARIOS      4338 non-null   float64
 14  data_hora_extracao     4400 non-null   object 
 15  DURA

In [24]:
for chave, coluna in enumerate(df_estatisticas_gerais_pandas.columns):
    print(chave, coluna)
    if chave in [0, 3,  6,  20, 7, 8]:
        df_estatisticas_gerais_pandas[coluna] = df_estatisticas_gerais_pandas[coluna].astype('string')
    elif chave in [1, 2, 5, 14, 19]:
        df_estatisticas_gerais_pandas[coluna] = df_estatisticas_gerais_pandas[coluna].astype('string')
    elif chave in [4, 5,10, 11, 12, 13, 16, 17, 18,  20]:
        df_estatisticas_gerais_pandas[coluna] = df_estatisticas_gerais_pandas[coluna].fillna(0).astype('int32')

0 data_extracao
1 DATA_PUBLICACAO
2 ID_CANAL
3 NM_CANAL
4 ID_CATEGORIA
5 ID_VIDEO
6 TITULO_VIDEO
7 DESCRICAO
8 TAGS
9 DURACAO_VIDEOS
10 TOTAL_VISUALIZACOES
11 TOTAL_LIKES
12 TOTAL_FAVORITOS
13 TOTAL_COMENTARIOS
14 data_hora_extracao
15 DURACAO_VIDEO_MINUTOS
16 TOTAL_CARACTERE_VIDEO
17 TOTAL_TAGS
18 HORA_EXTRACAO
19 TURNO_EXTRACAO
20 INDICE_TURNO_EXTRACAO


In [25]:
df_estatisticas_gerais_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_extracao          4400 non-null   string 
 1   DATA_PUBLICACAO        4400 non-null   string 
 2   ID_CANAL               4400 non-null   string 
 3   NM_CANAL               4400 non-null   string 
 4   ID_CATEGORIA           4400 non-null   int32  
 5   ID_VIDEO               4400 non-null   string 
 6   TITULO_VIDEO           4400 non-null   string 
 7   DESCRICAO              4400 non-null   string 
 8   TAGS                   3661 non-null   string 
 9   DURACAO_VIDEOS         4400 non-null   object 
 10  TOTAL_VISUALIZACOES    4400 non-null   int32  
 11  TOTAL_LIKES            4400 non-null   int32  
 12  TOTAL_FAVORITOS        4400 non-null   int32  
 13  TOTAL_COMENTARIOS      4400 non-null   int32  
 14  data_hora_extracao     4400 non-null   string 
 15  DURA

In [28]:
df_estatisticas_gerais_pandas[['ID_CATEGORIA','TOTAL_VISUALIZACOES', 'TOTAL_LIKES', 'TOTAL_COMENTARIOS']]

,ID_CATEGORIA,TOTAL_VISUALIZACOES,TOTAL_LIKES,TOTAL_COMENTARIOS
0,22,227954,7555,440
1,24,2278449,34934,624
2,20,227752,13706,955
3,20,2277398,140834,5644
4,10,22747,4719,4192
...,...,...,...,...
4395,20,228647,30397,937
4396,10,22835002,1322088,79948
4397,20,228218,5737,85
4398,24,228163,6259,106


In [29]:
df_estatisticas_gerais_pandas.to_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet')

In [30]:
spark.stop()